In [15]:
#Imports cell
#TODO clean this up
#this notebook is VERY ROUGH, but hopefully will one day apply U-net to a database of 3d ct scans
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image
from PIL import Image
import os
import tensorflow as tf
from keras.models import Sequential, Model 
from keras.layers import *
import keras.backend as k
import keras.utils
from keras import optimizers as opt
from sklearn.model_selection import train_test_split

print(k.image_dim_ordering())

k.set_image_dim_ordering('th')

#tf.keras

gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

th


C:\Users\PerkLab\Anaconda3\envs\DL1\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [16]:
#These paths are specific to my computer
#change them to match where you store your nrrds and segs
rootPath = "E:\\Patient Data Summer\\NN\\"
nrrdPath = "E:\\Patient Data Summer\\NN\\Nrrds\\"
segPath = "E:\\Patient Data Summer\\NN\\Segs\\"

nrrdFilePaths = []
nrrdFileNames = []
normalizedFilePaths = []
normalizedFileNames = []

labelFilePaths = []

#mainly unused, i could probably remove this
def loadNpy(name):
    b = np.load(name)
    return b

def bnormalize(arr):
    arrAbsolute = np.fabs(arr)
    arrMax = np.amax(arrAbsolute)
    print(arrAbsolute)
    print(arrMax)
    if arrMax != 0:
        arr = np.divide(arrAbsolute,arrMax)
        print("divided")
    else:
        print("error, max value is zero")
    return arr

for root, dirs, files in os.walk(nrrdPath):
        for filename in files:
            if filename.endswith("normalized.npy") or filename.endswith("5dim.npy"):
                continue
            elif filename.endswith(".npy"):
                path=os.path.join(root,filename)
                print(path)
                a = loadNpy(path)
                nrrdFilePaths.append(path)
                #very messy, but is necessary to weed out the normalized images
                if filename.endswith("normalized.npy") or filename.endswith("5dim.npy"):
                    continue
                elif filename.endswith(".npy"):
                    noExtension = filename[:-4]
                    nrrdFileNames.append(noExtension)
                    normalizedFileNames.append(noExtension)
                    newExtension = noExtension + '_normalized.npy'
                    outPutFileName = os.path.join(root,newExtension)
                    normalizedFilePaths.append(outPutFileName)
                

print(nrrdFilePaths)
print(normalizedFilePaths)
print(nrrdFileNames)

E:\Patient Data Summer\NN\Nrrds\PTMS0088_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTMS0088_Supine.npy
E:\Patient Data Summer\NN\Nrrds\PTMW0075_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTMW0075_Supine.npy
E:\Patient Data Summer\NN\Nrrds\PTOC0078_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTOC0078_Supine.npy
E:\Patient Data Summer\NN\Nrrds\PTOD0072_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTOD0072_Supine.npy
E:\Patient Data Summer\NN\Nrrds\PTPD0028_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTPD0028_Supine.npy
E:\Patient Data Summer\NN\Nrrds\PTPP0051_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTPP0051_Supine.npy
E:\Patient Data Summer\NN\Nrrds\PTPR0077_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTPR0077_Supine.npy
['E:\\Patient Data Summer\\NN\\Nrrds\\PTMS0088_Prone.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTMS0088_Supine.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTMW0075_Prone.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTMW0075_Supine.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTOC007

In [17]:
#get labels
#I dont normalize these because they are based on binary label maps
downsizedLabelFilePaths=[]
labelFilePaths=[]
for root, dirs, files in os.walk(segPath):
        for filename in files:
            if filename.endswith("Label.npy"):                
                noExtension = filename[:-4]
                newExtension = noExtension + '_downsized.npy'
                outPutFileName = os.path.join(root,newExtension)
                downsizedLabelFilePaths.append(outPutFileName)
                
                path=os.path.join(root,filename)
                labelFilePaths.append(path)
print(downsizedLabelFilePaths)

['E:\\Patient Data Summer\\NN\\Segs\\PTMS0088_ProSegLabel_downsized.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTMS0088_SupSegLabel_downsized.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTMW0075_ProSegLabel_downsized.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTMW0075_SupSegLabel_downsized.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTOC0078_ProSegLabel_downsized.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTOC0078_SupSegLabel_downsized.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTOD0072_ProSegLabel_downsized.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTOD0072_SupSegLabel_downsized.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTPD0028_ProSegLabel_downsized.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTPD0028_SupSegLabel_downsized.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTPP0051_ProSegLabel_downsized.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTPP0051_SupSegLabel_downsized.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTPR0077_ProSegLabel_downsized.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTPR0077_SupSe

In [18]:
#downsize CTs
zResize = []
from scipy.ndimage import zoom
nnnn=0
for i in nrrdFilePaths:
    print(i)
    a = loadNpy(i)
    a = zoom(a, (0.25,0.25,0.25))
    b=a.shape
    c=(b[0])
    d=128/c
    zResize.append(d)
    a=zoom(a, (d,1,1))
    print(a.shape)
    np.save(normalizedFilePaths[nnnn],a)
    nnnn+=1

E:\Patient Data Summer\NN\Nrrds\PTMS0088_Prone.npy
(128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTMS0088_Supine.npy


C:\Users\PerkLab\Anaconda3\envs\DL1\lib\site-packages\scipy\ndimage\interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


(128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTMW0075_Prone.npy
(128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTMW0075_Supine.npy
(128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTOC0078_Prone.npy
(128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTOC0078_Supine.npy
(128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTOD0072_Prone.npy
(128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTOD0072_Supine.npy
(128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTPD0028_Prone.npy
(128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTPD0028_Supine.npy
(128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTPP0051_Prone.npy
(128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTPP0051_Supine.npy
(128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTPR0077_Prone.npy
(128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTPR0077_Supine.npy
(128, 128, 128)


In [19]:
#downsize labels
nnnnn=0
for i in labelFilePaths:
    print(i)
    a = loadNpy(i)
    print(a.shape)
    a = zoom(a, (0.25,0.25,0.25))
    print(a.shape)
    b=a.shape
    c=(b[0])
    d=128/c
    print(d)
    zResize.append(d)
    a=zoom(a, (d,1,1))
    print(a.shape)
    np.save(downsizedLabelFilePaths[nnnnn],a)
    nnnnn+=1
    print(a)

E:\Patient Data Summer\NN\Segs\PTMS0088_ProSegLabel.npy
(733, 512, 512)
(183, 128, 128)
0.6994535519125683
(128, 128, 128)
[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]
E:\Patient Data Summer\NN\Segs\PTMS0088_SupSegLabel.npy
(678, 512, 512)
(170, 128, 128)
0.7529411764705

[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]
E:\Patient Data Summer\NN\Segs\PTPP0051_ProSegLabel.npy
(799, 512, 512)
(200, 128, 128)
0.64
(128, 128, 128)
[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ..

In [20]:
#create the normalized images
n=0
for i in normalizedFilePaths:
    print(i)
    a = loadNpy(i)
    b = bnormalize(a)
    np.save(normalizedFilePaths[n],b)
    n+=1
print(normalizedFilePaths)

E:\Patient Data Summer\NN\Nrrds\PTMS0088_Prone_normalized.npy
[[[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  ...
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]]

 [[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  ...
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]]

 [[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  ...
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]]

 ...

 [[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 204

E:\Patient Data Summer\NN\Nrrds\PTOD0072_Prone_normalized.npy
[[[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  ...
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]]

 [[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  ...
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]]

 [[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  ...
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]]

 ...

 [[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 204

E:\Patient Data Summer\NN\Nrrds\PTPP0051_Supine_normalized.npy
[[[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  ...
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]]

 [[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  ...
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]]

 [[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  ...
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]]

 ...

 [[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 20

In [21]:
print(labelFilePaths)

['E:\\Patient Data Summer\\NN\\Segs\\PTMS0088_ProSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTMS0088_SupSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTMW0075_ProSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTMW0075_SupSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTOC0078_ProSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTOC0078_SupSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTOD0072_ProSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTOD0072_SupSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTPD0028_ProSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTPD0028_SupSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTPP0051_ProSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTPP0051_SupSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTPR0077_ProSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTPR0077_SupSegLabel.npy']


In [22]:
#mostly borrowed from an article online
#i probably wont use this in the final version, this is mainly a placeholder for testing
smooth = 1.

def dice_coef(y_true, y_pred):
    y_true_f = k.flatten(y_true)
    y_pred_f = k.flatten(y_pred)
    intersection = k.sum(y_true_f * y_pred_f)
    
    return (2. * intersection + smooth) / (k.sum(y_true_f) + k.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    
    return -dice_coef(y_true, y_pred)

In [23]:
image5Dim = []
image5DimPaths = []
for yyy in normalizedFilePaths:
    print(yyy)
    a=np.load(yyy)
    a=a[np.newaxis,...]
    yyy = yyy[:-4]
    yyy=yyy + '_5dim.npy'
    np.save(yyy,a)
    print(np.shape(a))
    image5Dim.append(a)
    image5DimPaths.append(yyy)
    
print(np.shape(image5Dim))

E:\Patient Data Summer\NN\Nrrds\PTMS0088_Prone_normalized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTMS0088_Supine_normalized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTMW0075_Prone_normalized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTMW0075_Supine_normalized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTOC0078_Prone_normalized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTOC0078_Supine_normalized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTOD0072_Prone_normalized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTOD0072_Supine_normalized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTPD0028_Prone_normalized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTPD0028_Supine_normalized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTPP0051_Prone_normalized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Nrrds\PTPP0051_Supine_normalized.npy
(1, 128, 128, 128)
E:\Patient Data Summer

In [24]:
label5Dim = []
label5DimPaths = []

for iiiii in downsizedLabelFilePaths:
    print(iiiii)
    a=np.load(iiiii)
    a=a[np.newaxis,...]
    iiiii = iiiii[:-4]
    iiiii=iiiii + '_5dim.npy'
    np.save(iiiii,a)
    print(np.shape(a))
    label5Dim.append(a)
    label5DimPaths.append(yyy)
    
print(np.shape(label5Dim))

E:\Patient Data Summer\NN\Segs\PTMS0088_ProSegLabel_downsized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Segs\PTMS0088_SupSegLabel_downsized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Segs\PTMW0075_ProSegLabel_downsized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Segs\PTMW0075_SupSegLabel_downsized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Segs\PTOC0078_ProSegLabel_downsized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Segs\PTOC0078_SupSegLabel_downsized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Segs\PTOD0072_ProSegLabel_downsized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Segs\PTOD0072_SupSegLabel_downsized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Segs\PTPD0028_ProSegLabel_downsized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Segs\PTPD0028_SupSegLabel_downsized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Segs\PTPP0051_ProSegLabel_downsized.npy
(1, 128, 128, 128)
E:\Patient Data Summer\NN\Segs\PTPP0051_SupSegLabel_downsized.npy

In [25]:
#might not be necessary, but creates a list of the actual arrays, not just the paths
nn=0
listOfNormalized=[]
for i in normalizedFilePaths:  
    a=np.load(i)
    listOfNormalized.append(a)
    nn+=1
    print(np.shape(normalizedFilePaths))
print(listOfNormalized)

(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
[array([[[0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
         0.8797251],
        [0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
         0.8797251],
        [0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
         0.8797251],
        ...,
        [0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
         0.8797251],
        [0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
         0.8797251],
        [0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
         0.8797251]],

       [[0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
         0.8797251],
        [0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
         0.8797251],
        [0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
         0.8797251],
        ...,
        [0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
        

In [26]:
#same as above but for labels
#please excuse the variable names
nnn=0
listOfLabels=[]
for i in downsizedLabelFilePaths:  
    a=np.load(i)
    listOfLabels.append(a)
    nnn+=1
print(listOfLabels)

[array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ...,

In [42]:
X_train, X_test, y_train, y_test = train_test_split (image5Dim, label5Dim, test_size = 0.36, random_state = 0)


print(X_train)

print(np.shape(X_train))
print(np.shape(y_train))


[array([[[[0.88927484, 0.88927484, 0.88927484, ..., 0.88927484,
          0.88927484, 0.88927484],
         [0.88927484, 0.88927484, 0.88927484, ..., 0.88927484,
          0.88927484, 0.88927484],
         [0.88927484, 0.88927484, 0.88927484, ..., 0.88927484,
          0.88927484, 0.88927484],
         ...,
         [0.88927484, 0.88927484, 0.88927484, ..., 0.88927484,
          0.88927484, 0.88927484],
         [0.88927484, 0.88927484, 0.88927484, ..., 0.88927484,
          0.88927484, 0.88927484],
         [0.88927484, 0.88927484, 0.88927484, ..., 0.88927484,
          0.88927484, 0.88927484]],

        [[0.88927484, 0.88927484, 0.88927484, ..., 0.88927484,
          0.88927484, 0.88927484],
         [0.88927484, 0.88927484, 0.88927484, ..., 0.88927484,
          0.88927484, 0.88927484],
         [0.88927484, 0.88927484, 0.88927484, ..., 0.88927484,
          0.88927484, 0.88927484],
         ...,
         [0.88927484, 0.88927484, 0.88927484, ..., 0.88927484,
          0.88927484, 0.

In [43]:
#plt.imshow(rrrrrrrrrrrrrrr[0,0,50,0])
#plt.imshow(tttttttttttttt[90])

In [44]:
testInput1=np.load(normalizedFilePaths[0])
print(np.shape(testInput1))
testInput1=testInput1[np.newaxis,np.newaxis,...]
print(np.shape(testInput1))

testLabel1=np.load(downsizedLabelFilePaths[0])
print(np.shape(testLabel1))
testLabel1=testLabel1[np.newaxis,np.newaxis,...]

print(testInput1)

print(np.shape(testLabel1))

(128, 128, 128)
(1, 1, 128, 128, 128)
(128, 128, 128)
[[[[[0.8797251 0.8797251 0.8797251 ... 0.8797251 0.8797251 0.8797251]
    [0.8797251 0.8797251 0.8797251 ... 0.8797251 0.8797251 0.8797251]
    [0.8797251 0.8797251 0.8797251 ... 0.8797251 0.8797251 0.8797251]
    ...
    [0.8797251 0.8797251 0.8797251 ... 0.8797251 0.8797251 0.8797251]
    [0.8797251 0.8797251 0.8797251 ... 0.8797251 0.8797251 0.8797251]
    [0.8797251 0.8797251 0.8797251 ... 0.8797251 0.8797251 0.8797251]]

   [[0.8797251 0.8797251 0.8797251 ... 0.8797251 0.8797251 0.8797251]
    [0.8797251 0.8797251 0.8797251 ... 0.8797251 0.8797251 0.8797251]
    [0.8797251 0.8797251 0.8797251 ... 0.8797251 0.8797251 0.8797251]
    ...
    [0.8797251 0.8797251 0.8797251 ... 0.8797251 0.8797251 0.8797251]
    [0.8797251 0.8797251 0.8797251 ... 0.8797251 0.8797251 0.8797251]
    [0.8797251 0.8797251 0.8797251 ... 0.8797251 0.8797251 0.8797251]]

   [[0.8797251 0.8797251 0.8797251 ... 0.8797251 0.8797251 0.8797251]
    [0.8797251 0

In [45]:
print(np.shape(np.array(X_train)))
print(np.shape(y_train))
print(normalizedFilePaths)

(8, 1, 128, 128, 128)
(8, 1, 128, 128, 128)
['E:\\Patient Data Summer\\NN\\Nrrds\\PTMS0088_Prone_normalized.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTMS0088_Supine_normalized.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTMW0075_Prone_normalized.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTMW0075_Supine_normalized.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTOC0078_Prone_normalized.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTOC0078_Supine_normalized.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTOD0072_Prone_normalized.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTOD0072_Supine_normalized.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTPD0028_Prone_normalized.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTPD0028_Supine_normalized.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTPP0051_Prone_normalized.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTPP0051_Supine_normalized.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTPR0077_Prone_normalized.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTPR0077_Supine

In [51]:
#almost working fixed-layer implementation

NN = Model()
inp = Input((1,128,128,128))

#contracting path
conv1 = Conv3D(8, (3, 3, 3), activation='relu', padding='same')(inp)
conv1 = Conv3D(8, (3, 3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling3D(pool_size=(2, 2, 2))(conv1)
drop1 = Dropout(0.5)(pool1)

print(drop1.shape)
conv2 = Conv3D(16, (3, 3, 3), activation='relu', padding='same')(drop1)
conv2 = Conv3D(16, (3, 3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling3D(pool_size=(2, 2, 2))(conv2)
drop2 = Dropout(0.5)(pool2)
print(drop2.shape)
conv3 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(drop2)
conv3 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling3D(pool_size=(2, 2, 2))(conv3)
drop3 = Dropout(0.3)(pool3)
print(drop3.shape)
conv4 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(drop3)
conv4 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling3D(pool_size=(2, 2, 2))(conv4)
drop4 = Dropout(0.3)(pool4)
print(drop4.shape)
conv5 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(drop4)
conv5 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(conv5)
print(conv5.shape)
#expansion path
up6 = concatenate([Conv3DTranspose(64, (2, 2, 2), strides=2, padding='same')(conv5), conv4], axis=3)
conv6 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(up6)
conv6 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(conv6)
print(conv6.shape)
up7 = concatenate([Conv3DTranspose(32, (2, 2, 2), strides=2, padding='same')(conv6), conv3], axis=3)
conv7 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(up7)
conv7 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(conv7)
print(conv7.shape)
up8 = concatenate([Conv3DTranspose(16, (2, 2, 2), strides=2, padding='same')(conv7), conv2], axis=3)
conv8 = Conv3D(16, (3, 3, 3), activation='relu', padding='same')(up8)
conv8 = Conv3D(16, (3, 3, 3), activation='relu', padding='same')(conv8)
print(conv8.shape)
up9 = concatenate([Conv3DTranspose(8, (2, 2, 2), strides=2, padding='same')(conv8), conv1], axis=3)
conv9 = Conv3D(8, (3, 3, 3), activation='relu', padding='same')(up9)
conv9 = Conv3D(8, (3, 3, 3), activation='relu', padding='same')(conv9)
print(conv9.shape)
conv10 = Conv3D(1, (3, 3, 3), activation='softmax', padding='same')(conv9)
print(conv10.shape)
out = Model(inputs=[inp], outputs=[conv10])
out.compile(optimizer='adam', loss=dice_coef_loss, metrics=[dice_coef])
#out.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(testInput1.shape)
print(testLabel1.shape)

#print(testLabel1)

print(np.shape(X_train))

#new error when run on a machine with capable GPU:
'''
InvalidArgumentError: Incompatible shapes: [2097152] vs. [10128888]
	 [[{{node loss_8/conv3d_171_loss/mul}}, {{node metrics_8/dice_coef/mul}}]]
	 [[{{node loss_8/mul}}]]
'''
out.fit(np.array(X_train),np.array(y_train), batch_size=1, epochs=1, verbose=1)
print("fitted!")
#score = model.evaluate(x_test, y_test, batch_size=1)

#NN.summary()

#OOM FIT
#out.fit(testInput1,testLabel1, batch_size=1, epochs=1, verbose=1)

(?, 8, 64, 64, 64)
(?, 16, 32, 32, 32)
(?, 32, 16, 16, 16)
(?, 64, 8, 8, 8)
(?, 128, 8, 8, 8)
(?, 64, 16, ?, 16)
(?, 32, 32, ?, 32)
(?, 16, 64, ?, 64)
(?, 8, 128, ?, 128)
(?, 1, 128, ?, 128)
(1, 1, 128, 128, 128)
(1, 1, 128, 128, 128)
(8, 1, 128, 128, 128)
Epoch 1/1


InvalidArgumentError: Incompatible shapes: [2097152] vs. [10485760]
	 [[{{node loss_8/conv3d_171_loss/mul}}, {{node metrics_8/dice_coef/mul}}]]
	 [[{{node loss_8/mul}}]]

In [ ]:
#anything from this line down has not been tested, and is mainly experimental
ctSize=128

In [ ]:
#Adapted from segmentation2 to work for 3d
#not yet working

# Construct a U-net model
'''
num_classes = 2
filter_multiplier = 8

def nvidia_unet(patch_size=ctSize, num_classes=num_classes):
    input_ = Input((32, None, patch_size, patch_size))
    skips = []
    output = input_
    
    num_layers = int(np.floor(np.log2(patch_size)))
    down_conv_kernel_sizes = np.zeros([num_layers], dtype=int)
    down_filter_numbers = np.zeros([num_layers], dtype=int)
    up_conv_kernel_sizes = np.zeros([num_layers], dtype=int)
    up_filter_numbers = np.zeros([num_layers], dtype=int)
    
    for layer_index in range(num_layers):
        down_conv_kernel_sizes[layer_index] = int(3)
        down_filter_numbers[layer_index] = int( (layer_index + 1) * filter_multiplier + num_classes )
        up_conv_kernel_sizes[layer_index] = int(4)
        up_filter_numbers[layer_index] = int( (num_layers - layer_index - 1) * filter_multiplier + num_classes )
    
    print("Number of layers:       {}".format(num_layers))
    print("Filters in layers down: {}".format(down_filter_numbers))
    print("Filters in layers up:   {}".format(up_filter_numbers))
    
    for shape, filters in zip(down_conv_kernel_sizes, down_filter_numbers):
        skips.append(output)
        output= Conv3D(filters, (shape, shape, shape), strides=2, padding="same", activation="relu")(output)
    
    for shape, filters in zip(up_conv_kernel_sizes, up_filter_numbers):
        output = keras.layers.UpSampling3D(data_format='channels_first')(output)
        skip_output = skips.pop()
        print(output.shape)
        print(skip_output.shape)
        #the next line is throwing an error, not sure why
        output = concatenate([Conv3DTranspose(filters, (2,2,2), strides=2, padding='same')(output), skip_output], axis=3)
        if filters != num_classes:
            activation = "relu"
            output = Conv3D(filters, (shape, shape, shape), activation="relu", padding="same")(output)
            output = BatchNormalization(momentum=.9)(output)
        else:
            activation = "softmax"
            output = Conv3D(filters, (shape, shape, shape), activation="softmax", padding="same")(output)
    
    assert len(skips) == 0
    return Model([input_], [output])

model = nvidia_unet(ctSize, num_classes)

# model.summary()
'''

In [ ]:
'''
def create_unet_model3D(input_image_size,
                        n_labels=1,
                        layers=4,
                        lowest_resolution=16,
                        convolution_kernel_size=(3,3,3),
                        deconvolution_kernel_size=(3,3,3),
                        pool_size=(2,2,2),
                        strides=(2,2,2),
                        mode='classification',
                        output_activation='tanh',
                        init_lr=0.0001):
    
    #Create a 3D Unet model
    #Example
    #-------
    #unet_model = create_unet_model3D( (128,128,128,1), 1, 4)

    layers = np.arange(layers)
    number_of_classification_labels = n_labels
    
    inputs = Input(shape=input_image_size)

    ## ENCODING PATH ##

    encoding_convolution_layers = []
    pool = None
    for i in range(len(layers)):
        number_of_filters = lowest_resolution * 2**(layers[i])

        if i == 0:
            conv = Conv3D(filters=number_of_filters, 
                            kernel_size=convolution_kernel_size,
                            activation='relu',
                            padding='same')(inputs)
        else:
            conv = Conv3D(filters=number_of_filters, 
                            kernel_size=convolution_kernel_size,
                            activation='relu',
                            padding='same')(pool)

        encoding_convolution_layers.append(Conv3D(filters=number_of_filters, 
                                                        kernel_size=convolution_kernel_size,
                                                        activation='relu',
                                                        padding='same')(conv))

        if i < len(layers)-1:
            pool = MaxPooling3D(pool_size=pool_size)(encoding_convolution_layers[i])

    ## DECODING PATH ##
    outputs = encoding_convolution_layers[len(layers)-1]
    for i in range(1,len(layers)):
        number_of_filters = lowest_resolution * 2**(len(layers)-layers[i]-1)
        tmp_deconv = Conv3DTranspose(filters=number_of_filters, kernel_size=deconvolution_kernel_size,
                                     padding='same')(outputs)
        tmp_deconv = UpSampling3D(size=pool_size)(tmp_deconv)
        outputs = Concatenate(axis=4)([tmp_deconv, encoding_convolution_layers[len(layers)-i-1]])

        outputs = Conv3D(filters=number_of_filters, kernel_size=convolution_kernel_size, 
                        activation='relu', padding='same')(outputs)
        outputs = Conv3D(filters=number_of_filters, kernel_size=convolution_kernel_size, 
                        activation='relu', padding='same')(outputs)

    if mode == 'classification':
        if number_of_classification_labels == 1:
            outputs = Conv3D(filters=number_of_classification_labels, kernel_size=(1,1,1), 
                            activation='sigmoid')(outputs)
        else:
            outputs = Conv3D(filters=number_of_classification_labels, kernel_size=(1,1,1), 
                            activation='softmax')(outputs)

        unet_model = Model(inputs=inputs, outputs=outputs)

        if number_of_classification_labels == 1:
            unet_model.compile(loss=dice_coef_loss, 
                                optimizer=opt.Adam(lr=init_lr), metrics=[dice_coef])
        else:
            unet_model.compile(loss=dice_coef_loss, 
                                optimizer=opt.Adam(lr=init_lr), metrics=[dice_coef])
        
        #else:
            #unet_model.compile(loss='categorical_crossentropy', 
                                #optimizer=opt.Adam(lr=init_lr), metrics=['accuracy', 'categorical_crossentropy'])
        
    elif mode =='regression':
        outputs = Conv3D(filters=number_of_classification_labels, kernel_size=(1,1,1), 
                        activation=output_activation)(outputs)
        unet_model = Model(inputs=inputs, outputs=outputs)
        unet_model.compile(loss='mse', optimizer=opt.Adam(lr=init_lr))
    else:
        raise ValueError('mode must be either `classification` or `regression`')

    return unet_model
'''

In [ ]:
'''
bmodel = create_unet_model3D(input_image_size=(1,128, 128, 128), n_labels=2, 
                            layers=4, mode='classification')
'''

In [ ]:
testInput1=np.load(normalizedFilePaths[0])
print(np.shape(testInput1))
testInput1=testInput1[np.newaxis,np.newaxis,...]
print(np.shape(testInput1))

testLabel1=np.load(downsizedLabelFilePaths[0])
print(np.shape(testLabel1))
testLabel1=testLabel1[np.newaxis,np.newaxis,...]

print(testInput1)

print(np.shape(testLabel1))

max_learning_rate = 0.0001
min_learning_rate = 0.00001
num_epochs = 20

learning_rate_decay = (max_learning_rate - min_learning_rate) / num_epochs

bmodel.compile(optimizer=keras.optimizers.adam(lr=max_learning_rate, decay=learning_rate_decay),
               loss= "binary_crossentropy",
               metrics=["accuracy"])

print("Learning rate decay = {}".format(learning_rate_decay))

bmodel.fit(testInput1,testLabel1,epochs=num_epochs)


In [ ]:
# Batch Generator

import keras.utils
import scipy.ndimage

max_rotation_angle = 10

class UltrasoundSegmentationBatchGenerator(keras.utils.Sequence):
    
    def __init__(self,
                 x_set,
                 y_set,
                 batch_size,
                 image_dimensions=(None, 128, 128,1),
                 shuffle=True,
                 n_channels=1,
                 n_classes=2):
        self.x = x_set
        self.y = y_set
        self.batch_size = batch_size
        self.image_dimensions = image_dimensions
        self.shuffle = shuffle
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.number_of_images = self.x.shape[0]
        self.indexes = np.arange(self.number_of_images)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __len__(self):
        return int(np.floor(self.number_of_images / self.batch_size))
    
    def on_epoch_end(self):
        self.indexes = np.arange(self.number_of_images)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __getitem__(self, index):
        batch_indexes = self.indexes[index*self.batch_size : (index+1)*self.batch_size]
        x = np.empty((self.batch_size, *self.image_dimensions, self.n_channels))
        y = np.empty((self.batch_size, *self.image_dimensions))

        for i in range(self.batch_size):
            x[i,:,:,:,:] = self.x[batch_indexes[i],:,:,:,:]
            y[i,:,:,:]= self.y[batch_indexes[i],:,:,:]


        angle = np.random.randint(-max_rotation_angle, max_rotation_angle)
        x_rot = scipy.ndimage.interpolation.rotate(x, angle, (1,2), False, mode="constant", cval=0, order=0)
        y_rot = scipy.ndimage.interpolation.rotate(x, angle, (1,2), False, mode="constant", cval=0, order=0)

        x_rot = np.clip(x_rot, 0.0, 1.0)
        y_rot = np.clip(y_rot, 0.0, 1.0)
        
        y_onehot = keras.utils.to_categorical(y_rot, self.n_classes)

        return x_rot, y_onehot

In [ ]:
batch_size = 2

training_generator = UltrasoundSegmentationBatchGenerator(np.load(normalizedFilePaths[0]), np.load(labelFilePaths[0]), batch_size)
test_generator = UltrasoundSegmentationBatchGenerator(np.load(normalizedFilePaths[0]), np.load(labelFilePaths[1]), batch_size)

#training_time_start = datetime.datetime.now()

'''
training_log = model.fit_generator(training_generator,
                                   validation_data=test_generator,
                                   epochs=num_epochs,
                                   verbose=1)
                                   '''